In [224]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow import keras
import lightgbm as lgb
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor


In [225]:
def define_seed(seed_ = 42):

    import random
    import os
    os.environ["PYTHONHASHSEED"] = "42"
    os.environ["TF_DETERMINISTIC_OPS"] = "1"   # TF ≤ 2.12; harmless otherwise
    
    np.random.seed(seed_)
    tf.random.set_seed(seed_)
    random.seed(seed_)

define_seed()

In [226]:
def load_data():
    print("---------------- Load Data------------")

    #df = pd.read_parquet("C://fuel_consumption/last_data/interpolated_train_mean.parquet")
    df = pd.read_parquet("C://fuel_consumption/last_data/TRAIN_CLEAN_v2.parquet")
    #df = pd.read_parquet("C://fuel_consumption/train_segments_w_fuel.parquet")
    df_sub = pd.read_parquet("C://fuel_consumption/fuel_rank_submission.parquet")
    df_sub_all = pd.read_parquet("C://fuel_consumption/last_data/TRAIN_SUB_v2.parquet")

    print("\nData shape",df.shape,df_sub_all.shape,df_sub.shape)

    return df, df_sub, df_sub_all

df, df_sub, df_sub_all = load_data()

---------------- Load Data------------

Data shape (131530, 36) (24289, 36) (24289, 5)


In [227]:
Null_column=["mach_mean","mach_delta","engine_type"]

df.drop(Null_column, inplace=True, axis=1)
df_sub_all.drop(Null_column, inplace=True, axis=1)

In [228]:
len(df[df.flightphase_dominant=="NA"])

1371

In [229]:
len(df[df.flightphase_dominant=="LVL"])

2198

In [230]:
#df["fuel_rate"] = df["fuel_kg"] / df["segment_duration_s"].replace(0, np.nan)

#df["fuel_rate"].describe(percentiles=[0.99,0.995,0.999])


In [231]:
#rate_cap = df["fuel_rate"].quantile(0.999)

#df["bad_segment"] = (
#    (df["fuel_rate"] > rate_cap) |
##    (df["fuel_rate"] > 15) |                     # physics limit
#    (df["fuel_kg"] > df["fuel_kg"].quantile(0.999))  # extreme raw fuel
#)

#df = df[~df["bad_segment"]]


In [232]:
top_idx_cor =[44418,
 50775,
 82567,
 12347,
 70282,
 40461,
 42542,
 42543,
 42544,
 42545,
 42909,
 31795,
 35540,
 94135,
 15161,
 42139,
 3517,
 42910]
#df= df.drop(index=top_idx_cor).copy()

In [233]:
## Some feature enginnering
def feature_engineering(df, df_sub):

        ################## For training

    #df["altitude_mean"] =  df["altitude_mean"]*(-1.0)
    #df["fuel_kg"]= np.log1p(df["fuel_kg"])
    df["month"] = df["takeoff"].dt.month
    df["is_summer"] = df["month"].isin([6, 7, 8]).astype(int)
    df["size_class"] = df["typecode"].apply(map_aircraft_size)
    #df["flight_phase"] = df["altitude_delta"].apply(classify_flight_phase)
    #df["flight_phase"] = df.apply(lambda row: flight_phase(row["altitude_mean"], row["altitude_delta"]), axis=1)
    df["flight_phase"] = df.apply(
    lambda r: flight_phase(
        alt_delta=r["altitude_delta"],
        segment_cl=r["segment_cl"],
        segment_cr=r["segment_cr"],
        segment_de=r["segment_de"]
    ),axis=1)

    
    
    #df["start"] = pd.to_datetime(df["start"])
    #df["end"]   = pd.to_datetime(df["end"])
    
    #This to quantify  how each segment fuel burn differs from that flight overall average fuel burn.
    #df["fuel_diff_flight"] = df["fuel_kg"] - df.groupby("flight_id")["fuel_kg"].transform("mean") --> Problem on the submission file, we do not have target values

    df["altitude_mean_by_typecode"] = df.groupby("typecode")["altitude_mean"].transform("mean")

    #df["altitude_diff_type"] = df["altitude_mean"] - df.groupby("typecode")["altitude_mean"].transform("mean")
    
    df["altitude_diff_type"] = df["altitude_mean"] - df.groupby("typecode")["altitude_mean"].transform("mean")
    
    df["speed_diff_phase"] = df["groundspeed_mean"] - df.groupby("flight_phase")["groundspeed_mean"].transform("mean")

   # df["aircraft_generation"] = df["typecode"].apply(aircraft_generation)
    #df_sub["aircraft_generation"] = df_sub["typecode"].apply(aircraft_generation)
    #df_sub_all["aircraft_generation"] = df_sub_all["typecode"].apply(aircraft_generation)



    ### not tested yet 
    
    df["altitude_diff_flight"]   = df["altitude_mean"]   - df.groupby("flight_id")["altitude_mean"].transform("mean")
    #df["groundspeed_diff_flight"]= df["groundspeed_mean"]- df.groupby("flight_id")["groundspeed_mean"].transform("mean")
    #df["duration_diff_type"]     = df["duration_s"]        - df.groupby("typecode")["duration_s"].transform("mean")
    

    
    
    
    ############### For submisson ###############################
    df_sub["month"] = df_sub["takeoff"].dt.month
    df_sub["is_summer"] = df_sub["month"].isin([6, 7, 8]).astype(int)
    df_sub["size_class"] = df_sub["typecode"].apply(map_aircraft_size)
    
    #df_sub["flight_phase"] = df_sub["altitude_delta"].apply(classify_flight_phase)
    #df_sub["flight_phase"] = df_sub.apply(lambda row: flight_phase(row["altitude_mean"], row["altitude_delta"]), axis=1)


    #df_sub_all["start"] = pd.to_datetime(df_sub_all["start"])
    #df_sub_all["end"]   = pd.to_datetime(df_sub_all["end"])
    
    #df_sub["fuel_diff_flight"] = df_sub["fuel_kg"] - df_sub.groupby("flight_id")["fuel_kg"].transform("mean")
    df_sub["altitude_mean_by_typecode"] = df_sub.groupby("typecode")["altitude_mean"].transform("mean")
    
    df_sub["altitude_diff_type"] = df_sub["altitude_mean"] - df_sub.groupby("typecode")["altitude_mean"].transform("mean")

    df_sub["flight_phase"] = df_sub.apply(
    lambda r: flight_phase(
        alt_delta=r["altitude_delta"],
        segment_cl=r["segment_cl"],
        segment_cr=r["segment_cr"],
        segment_de=r["segment_de"]
    ),axis=1)

    
    
    df_sub["speed_diff_phase"] = df_sub["groundspeed_mean"] - df_sub.groupby("flight_phase")["groundspeed_mean"].transform("mean")

    df_sub["altitude_diff_flight"]   = df_sub["altitude_mean"]   - df_sub.groupby("flight_id")["altitude_mean"].transform("mean")
    #df_sub["groundspeed_diff_flight"]= df_sub["groundspeed_mean"]- df_sub.groupby("flight_id")["groundspeed_mean"].transform("mean")
    #df_sub["duration_diff_type"]     = df_sub["duration_s"]        - df_sub.groupby("typecode")["duration_s"].transform("mean")
 



    return df, df_sub


def map_aircraft_size1(typecode):
    large = {'B789','A359','B788','A332','A333','B77W','B772','B744','B763',
             'A388','B748','MD11','B77L','A306'}
    medium = {'A321','A320','A21N','A20N','B738','B739','B737','B752','B38M','B39M'}
    small = {'A319','A318'}

    if typecode in large:
        return 'large'
    elif typecode in medium:
        return 'medium'
    elif typecode in small:
        return 'small'
    else:
        return 'unknown'

def map_aircraft_size(typecode):
    huge = {'A388','B748','B744','MD11'}
    large = {'A359','B789','B788','A332','A333','B77W','B772',
             'B77L','B763','A306'}
    medium = {'A321','A320','A21N','A20N','A319','A318',
              'B738','B739','B737','B752','B38M','B39M'}

    if typecode in huge:
        return 'huge'
    elif typecode in large:
        return 'large'
    elif typecode in medium:
        return 'medium'
    else:
        return 'unknown'


def flight_phase1(alt_median, alt_delta):
    
    if alt_delta > 1000:
        return "climb"
    elif alt_delta < -1000:
        return "descent"
    
  
    if alt_median > 30000 and abs(alt_delta) <= 1000:
        return "cruise"
    
    return "level"

def flight_phase1(alt_delta,segment_cl, segment_cr, segment_de):

    # Climb
    if alt_delta > 1000 & segment_cr>0.5 :
        return "climb"

    # Descent
    elif alt_delta < -1000 & segment_de>0.5:
        return "descent"

    # Everything else = cruise
    elif segment_de>0.5:
        return "cruise"
    else:
        "cruise"

def flight_phase(alt_delta, segment_cl, segment_cr, segment_de,
                 alt_thresh=1000, frac_thresh=0.5):
    """
    Classify a segment into climb / cruise / descent
    using both altitude change and segment_* phase fractions.
    """

    # 1) Strong climb indicator: mostly climb AND going up,
    #    or very strong positive altitude change
    if (segment_cl >= frac_thresh and alt_delta > 0) or (alt_delta >= alt_thresh):
        return "climb"

    # 2) Strong descent indicator: mostly descent AND going down,
    #    or very strong negative altitude change
    if (segment_de >= frac_thresh and alt_delta < 0) or (alt_delta <= -alt_thresh):
        return "descent"

    # 3) If mostly cruise according to your segmentation → cruise
    if segment_cr >= frac_thresh:
        return "cruise"

    # 4) Fallbacks when fractions are ambiguous:
    #    use sign of altitude change
    if alt_delta > 0:
        return "climb"
    elif alt_delta < 0:
        return "descent"
    else:
        return "cruise"


def FE_climb(df_climb, df_sub_climb):
    
    climb_gs_mean_by_type = df_climb.groupby("typecode")["groundspeed_mean"].mean()
    df_climb["climb_gs_mean_type"]      = df_climb["typecode"].map(climb_gs_mean_by_type)
    df_sub_climb["climb_gs_mean_type"]      = df_sub_climb["typecode"].map(climb_gs_mean_by_type)

    return df_climb, df_sub_climb

def FE_cruise(df_climb, df_sub_climb):
    
    # 1) mean cruise altitude per type (TRAIN ONLY)
    cruise_alt_mean_by_type = df_climb.groupby("typecode")["altitude_mean"].mean()
    
    # 2) map to train + submission
    df_climb["cruise_alt_mean_type"]     = df_climb["typecode"].map(cruise_alt_mean_by_type)
    df_sub_climb["cruise_alt_mean_type"] = df_sub_climb["typecode"].map(cruise_alt_mean_by_type)


    return df_climb, df_sub_climb


def FE_descent(df, df_sub):
    
    # 1) mean cruise altitude per type (TRAIN ONLY)
    cruise_alt_mean_by_type = df.groupby("typecode")["altitude_mean"].mean()
    
    # 2) map to train + submission
    df["cruise_alt_mean_type"]     = df["typecode"].map(cruise_alt_mean_by_type)
    df_sub["cruise_alt_mean_type"] = df_sub["typecode"].map(cruise_alt_mean_by_type)


    #descent_vr_mean_by_type = df.groupby("typecode")["vertical_rate_mean"].mean()

    #df_climb["descent_vr_mean_type"]      = df_climb["typecode"].map(descent_vr_mean_by_type)
    #df_sub["descent_vr_mean_type"] = df_sub["typecode"].map(descent_vr_mean_by_type)



    return df, df_sub

new_gen = {
    "A21N","A20N","B38M","B39M","B789","B788","A359",
    "A388","B748","B77W","B77L"
}

old_gen = {"B744","MD11","A306"}

def aircraft_generation(typecode):
    if typecode in new_gen:
        return "new"
    elif typecode in old_gen:
        return "old"
    else:
        return "mid"



In [234]:
df, df_sub_all = feature_engineering(df, df_sub_all)
df.shape, df_sub_all.shape

((131530, 41), (24289, 41))

In [235]:
df

,flight_id,typecode,source,fuel_kg,dev_start_s,dev_end_s,wake,engine_number,total_flight_duration_s,takeoff,...,segment_na,flightphase_dominant,month,is_summer,size_class,flight_phase,altitude_mean_by_typecode,altitude_diff_type,speed_diff_phase,altitude_diff_flight
0,prc770822360,B789,acars,2500.0000,0.0,0.0,H,2,44297.0,2025-04-12 16:15:35,...,0.0,CR,4,0,large,cruise,35346.763713,641.856560,25.278564,1300.176891
1,prc770822360,B789,acars,1200.0000,0.0,0.0,H,2,44297.0,2025-04-12 16:15:35,...,0.0,CR,4,0,large,cruise,35346.763713,642.533457,35.702990,1300.853789
2,prc770822360,B789,acars,2500.0000,0.0,0.0,H,2,44297.0,2025-04-12 16:15:35,...,0.0,CR,4,0,large,cruise,35346.763713,642.264217,64.919070,1300.584548
3,prc770822360,B789,acars,1300.0000,0.0,0.0,H,2,44297.0,2025-04-12 16:15:35,...,0.0,CR,4,0,large,cruise,35346.763713,641.772689,59.976009,1300.093021
4,prc770822360,B789,acars,600.0000,0.0,0.0,H,2,44297.0,2025-04-12 16:15:35,...,0.0,DE,4,0,large,descent,35346.763713,-5860.028581,117.229590,-5201.708249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131525,prc806606087,A20N,acars,45.3592,0.0,0.0,M,2,9450.0,2025-08-31 23:29:32,...,0.0,CR,8,1,medium,cruise,30354.565800,7635.800140,13.094696,4693.715450
131526,prc806606087,A20N,acars,90.7184,0.0,0.0,M,2,9450.0,2025-08-31 23:29:32,...,0.0,CR,8,1,medium,cruise,30354.565800,7656.039595,9.820415,4713.954905
131527,prc806606087,A20N,acars,45.3592,0.0,0.0,M,2,9450.0,2025-08-31 23:29:32,...,0.0,CR,8,1,medium,cruise,30354.565800,7652.275366,7.568649,4710.190676
131528,prc806606087,A20N,acars,90.7184,0.0,0.0,M,2,9450.0,2025-08-31 23:29:32,...,0.0,DE,8,1,medium,descent,30354.565800,3717.306226,89.726604,775.221536


In [236]:
df.shape, df_sub_all.shape

((131530, 41), (24289, 41))

In [237]:
df.flight_phase.unique()

array(['cruise', 'descent', 'climb'], dtype=object)

In [238]:
#Climb phase: fuel varies widely → outliers may be real behavior -- Confirmed
#Cruise: stable → outliers usually errors/noise ---> confirmed
#Descent: low fuel usage → outliers more suspicious ------> confirmed

outliers_climb   = df.loc[top_idx_cor][df.loc[top_idx_cor].flight_phase == "climb"]
outliers_cruise  = df.loc[top_idx_cor][df.loc[top_idx_cor].flight_phase == "cruise"]
outliers_descent = df.loc[top_idx_cor][df.loc[top_idx_cor].flight_phase == "descent"]


In [239]:
outliers_climb.index

Index([82567, 40461, 35540, 42139, 3517], dtype='int64')

In [240]:

df=df.drop(outliers_cruise.index)
df=df.drop(outliers_descent.index)

In [164]:
### Devide the data by phases

In [241]:
df_climb   = df[df["flight_phase"] == "climb"].copy()
df_cruise  = df[df["flight_phase"] == "cruise"].copy()
df_descent = df[df["flight_phase"] == "descent"].copy()

In [242]:
df_sub_all_climb   = df_sub_all[df_sub_all["flight_phase"] == "climb"].copy()
df_sub_all_cruise  = df_sub_all[df_sub_all["flight_phase"] == "cruise"].copy()
df_sub_all_descent = df_sub_all[df_sub_all["flight_phase"] == "descent"].copy()

In [243]:
print("df: ", len(df_climb), len(df_cruise), len(df_descent))
print("df_sub_all:", len(df_sub_all_climb), len(df_sub_all_cruise), len(df_sub_all_descent))


df:  17276 94261 19980
df_sub_all: 3670 16483 4136


In [244]:
print("df: ", len(df_climb), len(df_cruise), len(df_descent))
print("df_sub_all:", len(df_sub_all_climb), len(df_sub_all_cruise), len(df_sub_all_descent))


df:  17276 94261 19980
df_sub_all: 3670 16483 4136


In [245]:
df.columns

Index(['flight_id', 'typecode', 'source', 'fuel_kg', 'dev_start_s',
       'dev_end_s', 'wake', 'engine_number', 'total_flight_duration_s',
       'takeoff', 'landing', 'segment_duration_s', 'segment_start_time_s',
       'segment_start_percentage', 'latitude_mean', 'latitude_delta',
       'longitude_mean', 'longitude_delta', 'altitude_mean', 'altitude_delta',
       'groundspeed_mean', 'groundspeed_delta', 'track_mean', 'track_delta',
       'vertical_rate_mean', 'vertical_rate_delta', 'segment_gnd',
       'segment_cl', 'segment_de', 'segment_lvl', 'segment_cr', 'segment_na',
       'flightphase_dominant', 'month', 'is_summer', 'size_class',
       'flight_phase', 'altitude_mean_by_typecode', 'altitude_diff_type',
       'speed_diff_phase', 'altitude_diff_flight'],
      dtype='object')

In [246]:
df_climb, df_sub_all_climb   = FE_climb(df_climb, df_sub_all_climb)
df_cruise, df_sub_all_cruise = FE_cruise(df_cruise, df_sub_all_cruise)
df_descent, df_sub_all_descent = FE_descent(df_descent, df_sub_all_descent)

In [247]:
df.shape, df_climb.shape, df_descent.shape

((131517, 41), (17276, 42), (19980, 42))

In [248]:
df_sub_all_climb.columns

Index(['flight_id', 'typecode', 'source', 'fuel_kg', 'dev_start_s',
       'dev_end_s', 'wake', 'engine_number', 'total_flight_duration_s',
       'takeoff', 'landing', 'segment_duration_s', 'segment_start_time_s',
       'segment_start_percentage', 'latitude_mean', 'latitude_delta',
       'longitude_mean', 'longitude_delta', 'altitude_mean', 'altitude_delta',
       'groundspeed_mean', 'groundspeed_delta', 'track_mean', 'track_delta',
       'vertical_rate_mean', 'vertical_rate_delta', 'segment_gnd',
       'segment_cl', 'segment_de', 'segment_lvl', 'segment_cr', 'segment_na',
       'flightphase_dominant', 'month', 'is_summer', 'size_class',
       'altitude_mean_by_typecode', 'altitude_diff_type', 'flight_phase',
       'speed_diff_phase', 'altitude_diff_flight', 'climb_gs_mean_type'],
      dtype='object')

In [249]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np

def split_by_flight_phase(df_phase, nan_cols, test_size=0.15, val_size=0.15, random_state=42):
    
    # 1) Unique flights in this phase
    flight_ids = df_phase["flight_id"].unique()

    # 2) Train/test split at flight level
    train_flights, test_flights = train_test_split(
        flight_ids, test_size=test_size, random_state=random_state, shuffle=True
    )

    # 3) Train/validation split
    val_ratio = val_size / (1 - test_size)
    
    train_flights, val_flights = train_test_split(
        train_flights, test_size=val_ratio, random_state=random_state, shuffle=True
    )

    # 4) Extract subsets
    train_df = df_phase[df_phase["flight_id"].isin(train_flights)].copy()
    val_df   = df_phase[df_phase["flight_id"].isin(val_flights)].copy()
    test_df  = df_phase[df_phase["flight_id"].isin(test_flights)].copy()

    # 5) Separate features/target
    y_train = train_df["fuel_kg"];  X_train = train_df.drop(columns=["fuel_kg","flight_id","segment_de","segment_cl","segment_lvl","segment_gnd","flight_phase"])
    y_val   = val_df["fuel_kg"];    X_val   = val_df.drop(columns=["fuel_kg","flight_id","segment_de","segment_cl","segment_lvl","segment_gnd","flight_phase"])
    y_test  = test_df["fuel_kg"];   X_test  = test_df.drop(columns=["fuel_kg","flight_id","segment_de","segment_cl","segment_lvl","segment_gnd","flight_phase"])

    # 6) Align columns
    X_val  = X_val.reindex(columns=X_train.columns, fill_value=np.nan)
    X_test = X_test.reindex(columns=X_train.columns, fill_value=np.nan)

    # 7) Handle NaNs
    imputer = SimpleImputer(strategy="mean")
    X_train[nan_cols] = imputer.fit_transform(X_train[nan_cols])
    X_val[nan_cols]   = imputer.transform(X_val[nan_cols])
    X_test[nan_cols]  = imputer.transform(X_test[nan_cols])
    

    
    
    
    return X_train, y_train, X_val, y_val, X_test, y_test, imputer


In [250]:
nan_cols = df.columns[df.isna().any()].tolist()

print("NaN columns:", nan_cols)


NaN columns: ['latitude_mean', 'latitude_delta', 'longitude_mean', 'longitude_delta', 'altitude_mean', 'altitude_delta', 'groundspeed_mean', 'groundspeed_delta', 'track_mean', 'track_delta', 'vertical_rate_mean', 'vertical_rate_delta', 'altitude_diff_type', 'speed_diff_phase', 'altitude_diff_flight']


In [251]:
####### Climb
X_train_climb, y_train_climb, \
X_val_climb, y_val_climb, \
X_test_climb, y_test_climb, imputer_climb = split_by_flight_phase(
    df_climb, nan_cols
)


In [252]:
#### Cruise
X_train_cruise, y_train_cruise, \
X_val_cruise, y_val_cruise, \
X_test_cruise, y_test_cruise, imputer_cruise = split_by_flight_phase(
    df_cruise, nan_cols
)


In [253]:
#### Descent
X_train_descent, y_train_descent, \
X_val_descent, y_val_descent, \
X_test_descent, y_test_descent, imputer_descent = split_by_flight_phase(
    df_descent, nan_cols
)


In [254]:
def compute_nan(df):
    a = df.isnull().sum()/(len(df))
    print (a)
compute_nan(X_val_cruise)

typecode                     0.0
source                       0.0
dev_start_s                  0.0
dev_end_s                    0.0
wake                         0.0
engine_number                0.0
total_flight_duration_s      0.0
takeoff                      0.0
landing                      0.0
segment_duration_s           0.0
segment_start_time_s         0.0
segment_start_percentage     0.0
latitude_mean                0.0
latitude_delta               0.0
longitude_mean               0.0
longitude_delta              0.0
altitude_mean                0.0
altitude_delta               0.0
groundspeed_mean             0.0
groundspeed_delta            0.0
track_mean                   0.0
track_delta                  0.0
vertical_rate_mean           0.0
vertical_rate_delta          0.0
segment_cr                   0.0
segment_na                   0.0
flightphase_dominant         0.0
month                        0.0
is_summer                    0.0
size_class                   0.0
altitude_m

In [255]:
X_train_climb.shape, y_train_climb.shape, X_val_climb.shape, y_val_climb.shape, X_test_climb.shape, y_test_climb.shape

((12042, 35), (12042,), (2602, 35), (2602,), (2632, 35), (2632,))

In [256]:
2602/(2602+12042+2632)

0.15061356795554526

In [257]:
cat_cols = X_train_climb.select_dtypes(include=['object', 'category']).columns.tolist()
print(cat_cols)

['typecode', 'source', 'wake', 'engine_number', 'flightphase_dominant', 'size_class']


In [258]:
#cat_cols.remove('flight_id')

from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    X_train_climb[col] = LabelEncoder().fit_transform(X_train_climb[col].astype(str))
    X_val_climb[col] = LabelEncoder().fit_transform(X_val_climb[col].astype(str))
    X_test_climb[col] = LabelEncoder().fit_transform(X_test_climb[col].astype(str))
    
    X_train_cruise[col] = LabelEncoder().fit_transform(X_train_cruise[col].astype(str))
    X_val_cruise[col] = LabelEncoder().fit_transform(X_val_cruise[col].astype(str))
    X_test_cruise[col] = LabelEncoder().fit_transform(X_test_cruise[col].astype(str))

    X_train_descent[col] = LabelEncoder().fit_transform(X_train_descent[col].astype(str))
    X_val_descent[col] = LabelEncoder().fit_transform(X_val_descent[col].astype(str))
    X_test_descent[col] = LabelEncoder().fit_transform(X_test_descent[col].astype(str))
    

In [259]:
#cat_cols = [ 'typecode', 'source', 'wake', 'engine_number',
      # 'flightphase_dominant', 'size_class', 'flight_phase']
from sklearn.preprocessing import LabelEncoder

for col in cat_cols:
    df_sub_all_climb[col] = LabelEncoder().fit_transform(df_sub_all_climb[col].astype(str))
    df_sub_all_cruise[col] = LabelEncoder().fit_transform(df_sub_all_cruise[col].astype(str))
    df_sub_all_descent[col] = LabelEncoder().fit_transform(df_sub_all_descent[col].astype(str))
    

In [260]:
Null_column=["takeoff","landing"]

X_train_climb.drop(Null_column, inplace=True, axis=1)
X_val_climb.drop(Null_column, inplace=True, axis=1)
X_test_climb.drop(Null_column, inplace=True, axis=1)

X_train_cruise.drop(Null_column, inplace=True, axis=1)
X_val_cruise.drop(Null_column, inplace=True, axis=1)
X_test_cruise.drop(Null_column, inplace=True, axis=1)

X_train_descent.drop(Null_column, inplace=True, axis=1)
X_val_descent.drop(Null_column, inplace=True, axis=1)
X_test_descent.drop(Null_column, inplace=True, axis=1)
#df_sub_all.drop(Null_column, inplace=True, axis=1)

In [261]:
Null_column=["takeoff","landing"]

df_sub_all_climb.drop(Null_column, inplace=True, axis=1)
df_sub_all_cruise.drop(Null_column, inplace=True, axis=1)
df_sub_all_descent.drop(Null_column, inplace=True, axis=1)
#df_sub_all.drop(Null_column, inplace=True, axis=1)

In [262]:
Nan_features_climb = X_train_climb.columns[X_train_climb.isna().any()].tolist()
if len(Nan_features_climb)>0:
    imputer_climb = SimpleImputer(strategy="mean")
    X_train_climb[Nan_features_climb] = imputer_climb.fit_transform(X_train_climb[Nan_features_climb])
    X_val_climb[Nan_features_climb]   = imputer_climb.transform(X_val_climb[Nan_features_climb])
    X_test_climb[Nan_features_climb]  = imputer_climb.transform(X_test_climb[Nan_features_climb])

    print("kaian hna climb")

Nan_features_cruise = X_train_cruise.columns[X_train_cruise.isna().any()].tolist()
if len(Nan_features_cruise)>0:
    imputer_cruise = SimpleImputer(strategy="mean")
    X_train_cruise[Nan_features_cruise] = imputer_cruise.fit_transform(X_train_cruise[Nan_features_cruise])
    X_val_cruise[Nan_features_cruise]   = imputer_cruise.transform(X_val_cruise[Nan_features_cruise])
    X_test_cruise[Nan_features_cruise]  = imputer_cruise.transform(X_test_cruise[Nan_features_cruise])
    print("kaian hna cruise")
    
Nan_features_descent = X_train_descent.columns[X_train_descent.isna().any()].tolist()
if len(Nan_features_descent)>0:
    imputer_descent = SimpleImputer(strategy="mean")
    X_train_descent[Nan_features_descent] = imputer_cruise.fit_transform(X_train_descent[Nan_features_descent])
    X_val_descent[Nan_features_descent]   = imputer_cruise.transform(X_val_descent[Nan_features_descent])
    X_test_descent[Nan_features_descent]  = imputer_cruise.transform(X_test_descent[Nan_features_descent])

    print("kaian hna descent")

In [263]:
Sub_Nan_features_climb = df_sub_all_climb.columns[df_sub_all_climb.isna().any()].tolist()
Sub_Nan_features_climb.remove("fuel_kg")

In [264]:
imputer_ = SimpleImputer(strategy="mean")
df_sub_all_climb[Sub_Nan_features_climb] = imputer_.fit_transform(df_sub_all_climb[Sub_Nan_features_climb])

In [265]:
Sub_Nan_features_descent = df_sub_all_descent.columns[df_sub_all_descent.isna().any()].tolist()
Sub_Nan_features_descent.remove("fuel_kg")

In [266]:
imputer__ = SimpleImputer(strategy="mean")
df_sub_all_descent[Sub_Nan_features_descent] = imputer__.fit_transform(df_sub_all_descent[Sub_Nan_features_descent])

In [267]:
Sub_Nan_features_cruise = df_sub_all_cruise.columns[df_sub_all_cruise.isna().any()].tolist()
Sub_Nan_features_cruise.remove("fuel_kg")

In [268]:
imputer___ = SimpleImputer(strategy="mean")
df_sub_all_cruise[Sub_Nan_features_cruise] = imputer___.fit_transform(df_sub_all_cruise[Sub_Nan_features_cruise])

In [269]:
if len(Nan_features_climb)>0:
    df_sub_all_climb[Nan_features_climb] = imputer_climb.fit_transform(df_sub_all_climb[Nan_features_climb])
    df_sub_all_cruise[Nan_features_cruise] = imputer_cruise.fit_transform(df_sub_all_cruise[Nan_features_cruise])
    df_sub_all_descent[Nan_features_descent] = imputer_descent.fit_transform(df_sub_all[Nan_features_descent])    

In [270]:
#mi = mutual_info_regression(x_train,y_train, random_state=42)
#mi_series = pd.Series(mi, index=x_train.columns).sort_values(ascending=False)
#print(mi_series)

In [271]:
#top_k = 30  # choose what you like
#top_features = mi_series.head(top_k).keys().tolist()

#print("Top features by mutual information:")
#print(top_features.)
#top_features.keys().tolist()

In [272]:
def normalize_data(x_train, x_val, x_test, x_sub):

    scaler = StandardScaler()
    #X_train = x_train.drop(columns=["flight_id"])
    #X_test   = x_test.drop(columns=["flight_id"])
    
    X_train_scaled = scaler.fit_transform(x_train)
    X_val_scaled   = scaler.transform(x_val)
    X_test_scaled   = scaler.transform(x_test)
    X_sub_scaled   = scaler.transform(x_sub)

    return X_train_scaled, X_val_scaled, X_test_scaled, X_sub_scaled, scaler

In [273]:
X_train_climb_scaled, X_val_climb_scaled, X_test_climb_scaled, X_sub_climb_scaled, scaler_climb = normalize_data (X_train_climb, X_val_climb, 
                                     X_test_climb, df_sub_all_climb[X_train_climb.columns])

In [274]:
X_train_cruise_scaled, X_val_cruise_scaled, X_test_cruise_scaled, X_sub_cruise_scaled, scaler_cruise = normalize_data (X_train_cruise, X_val_cruise, 
                                     X_test_cruise, df_sub_all_cruise[X_train_cruise.columns])

In [275]:
X_train_descent_scaled, X_val_descent_scaled, X_test_descent_scaled, X_sub_descent_scaled, scaler_descent = normalize_data (X_train_descent, X_val_descent, 
                                     X_test_descent, df_sub_all_descent[X_train_descent.columns])

In [276]:
df_climb.shape, df_sub_all_climb.shape

((17276, 42), (3670, 40))

# Stacking technique

In [277]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.base import clone
from sklearn.metrics import mean_absolute_error, mean_squared_error

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def stack_with_cv_and_test(X_train, y_train,X_test, y_test,base_models,meta_model,n_splits=5,random_state=42,shuffle=True):
    """
    Stage 1: use ONLY training data for CV stacking, then evaluate on test.

    X_train, y_train : training set used in K-fold
    X_test,  y_test  : held-out test set (never used in training folds)

    base_models : list of sklearn regressors
    meta_model  : sklearn regressor (e.g. Ridge)

    Returns
    -------
    meta_fitted    : meta model fitted on OOF level-1 features (from X_train)
    oof_level1     : OOF level-1 features for X_train
    test_pred      : final stacked predictions on X_test
    (mae_test, rmse_test) : test metrics
    """

    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
    X_test = np.asarray(X_test)
    y_test = np.asarray(y_test)

    n_train = X_train.shape[0]
    n_test = X_test.shape[0]
    n_models = len(base_models)

    oof_level1 = np.zeros((n_train, n_models))
    test_level1 = np.zeros((n_test, n_models))

    kf = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    print("\n===== STAGE 1: CV STACKING ON TRAIN, EVAL ON TEST =====")

    for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train, y_train), 1):
        print(f"\n----- Fold {fold}/{n_splits} -----")
        X_tr, X_val = X_train[tr_idx], X_train[val_idx]
        y_tr, y_val = y_train[tr_idx], y_train[val_idx]

        for m_idx, base in enumerate(base_models):
            model = clone(base)
            model.fit(X_tr, y_tr)

            # OOF preds on val part of TRAIN
            val_pred = model.predict(X_val)
            oof_level1[val_idx, m_idx] = val_pred

            # Accumulate preds on TEST
            test_level1[:, m_idx] += model.predict(X_test) / n_splits

            mae_f = mean_absolute_error(y_val, val_pred)
            rmse_f = rmse(y_val, val_pred)
            print(f"  Model {m_idx+1}: MAE={mae_f:.3f}, RMSE={rmse_f:.3f}")

    # Fit meta-model on OOF level-1 features
    meta_fitted = clone(meta_model)
    meta_fitted.fit(oof_level1, y_train)

    # OOF performance of stacked model (CV estimate)
    oof_meta_pred = meta_fitted.predict(oof_level1)
    mae_oof = mean_absolute_error(y_train, oof_meta_pred)
    rmse_oof = rmse(y_train, oof_meta_pred)

    print("\n===== STACKED OOF PERFORMANCE (TRAIN) =====")
    print(f"OOF MAE:  {mae_oof:.3f}")
    print(f"OOF RMSE: {rmse_oof:.3f}")

    # Final stacked preds on TEST
    test_meta_pred = meta_fitted.predict(test_level1)
    mae_test = mean_absolute_error(y_test, test_meta_pred)
    rmse_test = rmse(y_test, test_meta_pred)

    print("\n===== STACKED TEST PERFORMANCE =====")
    print(f"TEST MAE:  {mae_test:.3f}")
    print(f"TEST RMSE: {rmse_test:.3f}")

    return meta_fitted, oof_level1, test_meta_pred, (mae_test, rmse_test)



import numpy as np
from sklearn.model_selection import KFold
from sklearn.base import clone
from sklearn.metrics import mean_absolute_error, mean_squared_error

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def stack_for_submission(
    X, y,
    X_sub,
    base_models,
    meta_model,
    n_splits=5,
    random_state=42,
    shuffle=True,
):
    """
    Stage 2: Use ALL labeled data (train + test combined) for CV stacking,
    train meta-model on OOF predictions, and predict on submission data.

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        Full labeled dataset = X_train + X_test (after you are satisfied with test performance)

    y : array-like, shape (n_samples,)
        Full target values = y_train + y_test

    X_sub : array-like, shape (n_sub, n_features)
        Final submission data (no labels)

    base_models : list
        List of sklearn regressors (e.g., [lgbm, gbdt])

    meta_model : sklearn regressor
        Final estimator to stack on top (e.g., Ridge)

    Returns
    -------
    meta_fitted : trained final meta-model
    oof_level1  : OOF predictions from base models on full labeled data
    sub_level1  : base models' predictions on submission data, averaged across folds
    sub_pred    : final stacked prediction for submission
    """

    # Convert to arrays
    X = np.asarray(X)
    y = np.asarray(y)
    X_sub = np.asarray(X_sub)

    n_samples = X.shape[0]
    n_sub = X_sub.shape[0]
    n_models = len(base_models)

    # Level-1 matrices
    oof_level1 = np.zeros((n_samples, n_models))
    sub_level1 = np.zeros((n_sub, n_models))

    # CV
    kf = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    print("\n===== STAGE 2: FINAL STACKING ON ALL LABELED DATA =====")

    # K-fold stacking loop
    for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n----- Fold {fold}/{n_splits} -----")
        X_tr, X_val = X[tr_idx], X[val_idx]
        y_tr, y_val = y[tr_idx], y[val_idx]

        for m_idx, base in enumerate(base_models):
            model = clone(base)
            model.fit(X_tr, y_tr)

            # OOF predictions
            val_pred = model.predict(X_val)
            oof_level1[val_idx, m_idx] = val_pred

            # Predictions on submission data (averaged across folds)
            sub_level1[:, m_idx] += model.predict(X_sub) / n_splits

            # Per-fold metrics
            mae_f = mean_absolute_error(y_val, val_pred)
            rmse_f = rmse(y_val, val_pred)
            print(f"  Model {m_idx+1}: MAE={mae_f:.3f}, RMSE={rmse_f:.3f}")

    # Train meta-model on the final OOF matrix
    meta_fitted = clone(meta_model)
    meta_fitted.fit(oof_level1, y)

    # OOF performance of stacked meta-model
    oof_meta_pred = meta_fitted.predict(oof_level1)
    mae_oof = mean_absolute_error(y, oof_meta_pred)
    rmse_oof = rmse(y, oof_meta_pred)

    print("\n===== STACKED OOF PERFORMANCE (ALL LABELED DATA) =====")
    print(f"OOF MAE:  {mae_oof:.3f}")
    print(f"OOF RMSE: {rmse_oof:.3f}")

    # Final stacked predictions for submission
    sub_pred = meta_fitted.predict(sub_level1)

    return meta_fitted, oof_level1, sub_level1, sub_pred



## Climb phase

In [278]:
X_trainVal_climb = np.vstack([X_train_climb_scaled, X_val_climb_scaled])
y_trainVal_climb = np.concatenate([y_train_climb, y_val_climb])

In [280]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.ensemble import ExtraTreesRegressor


lgb_params_climb_base = dict(
    objective="regression",
    boosting_type="dart",   # or "gbdt" to compare
    num_leaves=8,
    max_depth=5,
    min_data_in_leaf=10,
    learning_rate=0.02,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    n_estimators=5000,
    random_state=42,
    n_jobs=-1,
    verbose=-1,
)
lgbm_base_climb = LGBMRegressor(**lgb_params_climb_base)

gbdt_base_climb = GradientBoostingRegressor(n_estimators=300,learning_rate=0.02,max_depth=5,random_state=42)

#Rf_base_climb = RandomForestRegressor(n_estimators=200, max_depth=5)
xgb_base_climb = XGBRegressor(max_depth=6, n_estimators=200, random_state=42)

Xtrees_base_climb = ExtraTreesRegressor(n_estimators=500,max_depth=None, random_state=42)

base_models_climb = [lgbm_base_climb, gbdt_base_climb,xgb_base_climb,Xtrees_base_climb]

meta_model_climb = LinearRegression()#Ridge(alpha=1.0)#RandomForestRegressor(max_depth=4)   # you can tune alpha later

In [281]:
meta_eval, oof_train, test_pred, test_metrics = stack_with_cv_and_test(
    X_trainVal_climb, y_trainVal_climb,
    X_test_climb_scaled, y_test_climb,
    base_models_climb, meta_model_climb,
    n_splits=5,
    random_state=42)


===== STAGE 1: CV STACKING ON TRAIN, EVAL ON TEST =====

----- Fold 1/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=112.047, RMSE=281.365
  Model 2: MAE=121.592, RMSE=268.027
  Model 3: MAE=113.066, RMSE=294.656
  Model 4: MAE=102.739, RMSE=267.533

----- Fold 2/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=111.178, RMSE=305.731
  Model 2: MAE=120.620, RMSE=332.001
  Model 3: MAE=118.139, RMSE=334.903
  Model 4: MAE=103.656, RMSE=316.123

----- Fold 3/5 -----


...

  Model 1: MAE=118.815, RMSE=352.630
  Model 2: MAE=124.464, RMSE=324.790
  Model 3: MAE=121.412, RMSE=344.575
  Model 4: MAE=109.409, RMSE=343.222

----- Fold 4/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=114.545, RMSE=298.455
  Model 2: MAE=124.498, RMSE=329.244
  Model 3: MAE=118.996, RMSE=329.984
  Model 4: MAE=106.485, RMSE=302.952

----- Fold 5/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=116.744, RMSE=292.892
  Model 2: MAE=125.022, RMSE=291.002
  Model 3: MAE=118.528, RMSE=293.183
  Model 4: MAE=106.666, RMSE=296.772

===== STACKED OOF PERFORMANCE (TRAIN) =====
OOF MAE:  107.824
OOF RMSE: 294.763

===== STACKED TEST PERFORMANCE =====
TEST MAE:  106.085
TEST RMSE: 222.793


In [ ]:
TEST MAE:  107.321
TEST RMSE: 222.781

In [58]:
#################### Submission ######################
######################################################

In [59]:
X_full_climb = np.vstack([X_train_climb_scaled, X_val_climb_scaled, X_test_climb_scaled])
y_full_climb = np.concatenate([y_train_climb, y_val_climb, y_test_climb])

In [60]:
meta_final_climb, oof_full_climb, sub_level1_climb, sub_pred_climb = stack_for_submission(
    X=X_full_climb,
    y=y_full_climb,
    X_sub=X_sub_climb_scaled,
    base_models=base_models_climb,
    meta_model=meta_model_climb,
    n_splits=5,
    random_state=42)



===== STAGE 2: FINAL STACKING ON ALL LABELED DATA =====

----- Fold 1/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=115.623, RMSE=276.715
  Model 2: MAE=119.224, RMSE=292.759
  Model 3: MAE=112.093, RMSE=290.090
  Model 4: MAE=99.997, RMSE=274.876

----- Fold 2/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=119.128, RMSE=306.220
  Model 2: MAE=123.416, RMSE=323.402
  Model 3: MAE=116.800, RMSE=321.187
  Model 4: MAE=107.663, RMSE=324.949

----- Fold 3/5 -----


...

  Model 1: MAE=124.073, RMSE=428.961
  Model 2: MAE=127.077, RMSE=419.597
  Model 3: MAE=122.148, RMSE=424.509
  Model 4: MAE=111.450, RMSE=406.516

----- Fold 4/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=119.003, RMSE=280.205
  Model 2: MAE=122.315, RMSE=311.853
  Model 3: MAE=115.791, RMSE=313.524
  Model 4: MAE=104.993, RMSE=292.786

----- Fold 5/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=116.805, RMSE=275.743
  Model 2: MAE=119.420, RMSE=278.661
  Model 3: MAE=115.084, RMSE=302.027
  Model 4: MAE=103.147, RMSE=286.974

===== STACKED OOF PERFORMANCE (ALL LABELED DATA) =====
OOF MAE:  106.464
OOF RMSE: 310.586


In [ ]:
OOF MAE:  105.975
OOF RMSE: 310.159

In [61]:
sub_pred_climb

array([ 674.72354617,  170.68320743,  216.48889478, ...,   97.15460946,
        922.39706267, 1105.4846279 ])

In [98]:
sub_pred_climb

array([ 655.65472897,  169.67018975,  219.8727815 , ...,   99.41806042,
        934.85263501, 1126.00685394])

In [ ]:
######################################################
######################################################

## Cruise phase

In [62]:
X_trainVal_cruise = np.vstack([X_train_cruise_scaled, X_val_cruise_scaled])
y_trainVal_cruise = np.concatenate([y_train_cruise, y_val_cruise])

In [63]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge,LinearRegression

lgb_params_cruise_base = dict(
    objective="regression",
    boosting_type="dart",   # or "gbdt" to compare
    num_leaves=20,
    max_depth=6,
    min_data_in_leaf=50,
    learning_rate=0.02,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    n_estimators=5000,
    random_state=42,
    n_jobs=-1,
    verbose=-1,
)
lgbm_base_cruise = LGBMRegressor(**lgb_params_cruise_base)

gbdt_base_cruise = GradientBoostingRegressor(n_estimators=500, learning_rate=0.02, max_depth=6,random_state=42)

Rf_base_cruise = RandomForestRegressor(n_estimators=200, max_depth=5)
Xtrees_base_cruise = ExtraTreesRegressor(n_estimators=500,max_depth=None, random_state=42)

base_models_cruise = [lgbm_base_cruise, gbdt_base_cruise, Xtrees_base_cruise]

meta_model_cruise = LinearRegression()#Ridge(alpha=1.0)#RandomForestRegressor(max_depth=4)   # you can tune alpha later

In [101]:
meta_eval_cruise, oof_train_cruise, test_pred_cruise, test_metrics_cruise = stack_with_cv_and_test(
    X_trainVal_cruise, y_trainVal_cruise,
    X_test_cruise_scaled, y_test_cruise,
    base_models_cruise, meta_model_cruise,
    n_splits=5,
    random_state=42)


===== STAGE 1: CV STACKING ON TRAIN, EVAL ON TEST =====

----- Fold 1/5 -----


KeyboardInterrupt: 

In [ ]:
TEST MAE:  83.055
TEST RMSE: 216.077

In [ ]:
############################### Submission ###############################
##########################################################################

In [64]:
X_full_cruise = np.vstack([X_train_cruise_scaled, X_val_cruise_scaled, X_test_cruise_scaled])
y_full_cruise = np.concatenate([y_train_cruise, y_val_cruise, y_test_cruise])

In [65]:
meta_final_cruise, oof_full_cruise, sub_level1_cruise, sub_pred_cruise = stack_for_submission(
    X=X_full_cruise,
    y=y_full_cruise,
    X_sub=X_sub_cruise_scaled,
    base_models=base_models_cruise,
    meta_model=meta_model_cruise,
    n_splits=5,
    random_state=42)



===== STAGE 2: FINAL STACKING ON ALL LABELED DATA =====

----- Fold 1/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=72.960, RMSE=185.092
  Model 2: MAE=71.579, RMSE=190.721
  Model 3: MAE=67.017, RMSE=189.254

----- Fold 2/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=73.164, RMSE=180.133
  Model 2: MAE=71.153, RMSE=176.299
  Model 3: MAE=65.786, RMSE=172.440

----- Fold 3/5 -----


...

  Model 1: MAE=71.791, RMSE=176.956
  Model 2: MAE=69.939, RMSE=172.540
  Model 3: MAE=65.251, RMSE=170.396

----- Fold 4/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=71.228, RMSE=173.424
  Model 2: MAE=69.337, RMSE=167.943
  Model 3: MAE=64.047, RMSE=161.730

----- Fold 5/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=71.048, RMSE=189.737
  Model 2: MAE=69.748, RMSE=191.406
  Model 3: MAE=64.542, RMSE=187.387

===== STACKED OOF PERFORMANCE (ALL LABELED DATA) =====
OOF MAE:  65.335
OOF RMSE: 171.773


In [81]:
sub_pred_cruise

array([190.06121968, 189.99830809, 190.94676586, ...,  67.93643039,
        99.48632973, 112.74493304])

In [91]:
sub_pred_cruise

array([186.54863672, 183.63872763, 187.42446776, ...,  70.01720445,
       101.54705717,  92.56219483])

## Descent phase

In [66]:
X_trainVal_descent = np.vstack([X_train_descent_scaled, X_val_descent_scaled])
y_trainVal_descent = np.concatenate([y_train_descent, y_val_descent])

In [67]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge,LinearRegression

lgb_params_descent_base = dict(
    objective="regression",
    boosting_type="dart",   # or "gbdt" to compare
    num_leaves=15,
    max_depth=5,
    min_data_in_leaf=20,
    learning_rate=0.02,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    n_estimators=5000,
    random_state=42,
    n_jobs=-1,
    verbose=-1)
lgbm_base_descent = LGBMRegressor(**lgb_params_descent_base)

gbdt_base_descent = GradientBoostingRegressor(n_estimators=300, learning_rate=0.02, max_depth=5,random_state=42)

Rf_base_descent = RandomForestRegressor(n_estimators=200, max_depth=5)
base_models_descent = [lgbm_base_descent, gbdt_base_descent]

meta_model_descent = LinearRegression()#Ridge(alpha=1.0)#RandomForestRegressor(max_depth=4)   # you can tune alpha later

In [164]:
meta_eval_descent, oof_train_descent, test_pred_descent, test_metrics_descent = stack_with_cv_and_test(
    X_trainVal_descent, y_trainVal_descent,
    X_test_descent_scaled, y_test_descent,
    base_models_descent, meta_model_descent,
    n_splits=5,
    random_state=42)


===== STAGE 1: CV STACKING ON TRAIN, EVAL ON TEST =====

----- Fold 1/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=67.645, RMSE=218.208
  Model 2: MAE=69.187, RMSE=219.079

----- Fold 2/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=69.526, RMSE=199.712
  Model 2: MAE=70.158, RMSE=200.245

----- Fold 3/5 -----


...

  Model 1: MAE=66.237, RMSE=166.654
  Model 2: MAE=68.095, RMSE=163.025

----- Fold 4/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=59.747, RMSE=164.856
  Model 2: MAE=63.597, RMSE=165.094

----- Fold 5/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=70.555, RMSE=195.991
  Model 2: MAE=71.589, RMSE=190.217

===== STACKED OOF PERFORMANCE (TRAIN) =====
OOF MAE:  65.220
OOF RMSE: 182.503

===== STACKED TEST PERFORMANCE =====
TEST MAE:  74.593
TEST RMSE: 217.727


In [ ]:
217.727

In [ ]:
############################ Submission ####################################
############################################################################

In [68]:
X_full_descent = np.vstack([X_train_descent_scaled, X_val_descent_scaled, X_test_descent_scaled])
y_full_descent = np.concatenate([y_train_descent, y_val_descent, y_test_descent])

In [69]:
lgb_params_descent_base = dict(
    objective="regression",
    boosting_type="dart",   # or "gbdt" to compare
    num_leaves=15,
    max_depth=5,
    min_data_in_leaf=20,
    learning_rate=0.02,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    n_estimators=5000,
    random_state=42,
    n_jobs=-1,
    verbose=-1)
lgbm_base_descent = LGBMRegressor(**lgb_params_descent_base)

gbdt_base_descent = GradientBoostingRegressor(n_estimators=300, learning_rate=0.02, max_depth=5,random_state=42)

Rf_base_descent = RandomForestRegressor(n_estimators=200, max_depth=5)
Xtrees_base_descent = ExtraTreesRegressor(n_estimators=500,max_depth=None, random_state=42)

base_models_descent = [lgbm_base_descent, gbdt_base_descent, Xtrees_base_descent]
meta_model_descent = LinearRegression()#Ridge(alpha=1.0)#RandomForestRegressor(max_depth=4)   # you can tune alpha later

In [70]:
meta_final_descent, oof_full_descent, sub_level1_descent, sub_pred_descent = stack_for_submission(
    X=X_full_descent,
    y=y_full_descent,
    X_sub=X_sub_descent_scaled,
    base_models=base_models_descent,
    meta_model=meta_model_descent,
    n_splits=5,
    random_state=42)



===== STAGE 2: FINAL STACKING ON ALL LABELED DATA =====

----- Fold 1/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=64.111, RMSE=200.024
  Model 2: MAE=66.116, RMSE=187.839
  Model 3: MAE=58.672, RMSE=188.566

----- Fold 2/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=69.869, RMSE=205.025
  Model 2: MAE=71.712, RMSE=213.670
  Model 3: MAE=65.074, RMSE=215.257

----- Fold 3/5 -----


...

  Model 1: MAE=63.300, RMSE=192.547
  Model 2: MAE=68.510, RMSE=212.644
  Model 3: MAE=59.754, RMSE=191.982

----- Fold 4/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=69.840, RMSE=182.858
  Model 2: MAE=73.677, RMSE=185.816
  Model 3: MAE=65.164, RMSE=165.673

----- Fold 5/5 -----


C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhiat\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


  Model 1: MAE=67.705, RMSE=174.086
  Model 2: MAE=70.447, RMSE=169.367
  Model 3: MAE=63.099, RMSE=173.773

===== STACKED OOF PERFORMANCE (ALL LABELED DATA) =====
OOF MAE:  62.301
OOF RMSE: 182.519


In [71]:
sub_pred_descent

array([149.79019825,  99.88094096,  96.72364402, ..., 146.77133677,
        99.83352015, 195.79436129])

In [ ]:
############################################################################

In [108]:
sub_pred_descent

array([148.12559212,  99.7123919 ,  97.43882619, ..., 146.49900841,
       100.35065156, 195.93570245])

In [ ]:
###### Combine all the submission from differernt phasess and create the submission file

In [72]:
len(sub_pred_climb) + len(sub_pred_cruise) + len(sub_pred_descent)

24289

In [73]:
import numpy as np
import pandas as pd

final_sub = df_sub.copy()

if "fuel_kg" not in final_sub.columns:
    final_sub["fuel_kg"] = 0.0

final_sub.loc[df_sub_all_climb.index, "fuel_kg"] = sub_pred_climb.reshape(-1)

final_sub.loc[df_sub_all_cruise.index, "fuel_kg"] = sub_pred_cruise.reshape(-1)

# Fill in descent predictions
final_sub.loc[df_sub_all_descent.index, "fuel_kg"] = sub_pred_descent.reshape(-1)


In [74]:
final_sub.to_csv("C://fuel_consumption/unique-honey_v56.csv", index=False)
final_sub.to_parquet("C://fuel_consumption/unique-honey_v56.parquet", index=False)